In [ ]:
from sklearn.model_selection import StratifiedKFold

n_splits = 10

skf = StratifiedKFold(n_splits=n_splits, random_state=42, shuffle=True)

i =0
for train_idx, test_idx in (skf.split(FDS_data, FDS_Tg)):
  
  globals()['train_feat_'+str(i+1)] = FDS_data.iloc[train_idx]
  globals()['train_label_'+str(i+1)] = FDS_Tg.iloc[train_idx]

  globals()['valid_feat_'+str(i+1)] = FDS_data.iloc[test_idx]
  globals()['valid_label_'+str(i+1)] = FDS_Tg.iloc[test_idx]
  i=i+1

In [ ]:
for i in range(n_splits):
  print( len(globals()['train_feat_'+str(i+1)]), len(globals()['train_label_'+str(i+1)]),
        len(globals()['valid_feat_'+str(i+1)]), len(globals()['valid_label_'+str(i+1)]) )

In [ ]:
for i in range(n_splits):
  print(i+1)
  globals()['h2train_feat_'+str(i+1)] = h2o.H2OFrame(globals()['train_feat_'+str(i+1)], column_names=list(FDS_data.columns))
  globals()['h2train_target_'+str(i+1)] = h2o.H2OFrame(pd.DataFrame(globals()['train_label_'+str(i+1)]), column_names=['TARGET'])

  globals()['h2valid_feat_'+str(i+1)] = h2o.H2OFrame(globals()['valid_feat_'+str(i+1)], column_names=list(FDS_data.columns))
  globals()['h2valid_target_'+str(i+1)] = h2o.H2OFrame(pd.DataFrame(globals()['valid_label_'+str(i+1)]), column_names=['TARGET'])

In [ ]:
from tqdm import tqdm
for i in tqdm(range(n_splits)):
  globals()['h2train_xy_'+str(i+1)] = globals()['h2train_feat_'+str(i+1)].cbind(globals()['h2train_target_'+str(i+1)])
  globals()['h2valid_xy_'+str(i+1)] = globals()['h2valid_feat_'+str(i+1)].cbind(globals()['h2valid_target_'+str(i+1)])

In [ ]:
from h2o.estimators import H2OXGBoostEstimator
import optuna
from optuna import Trial, visualization
import time

def Objective(trial):
    # 일반적인 Xgb 와 H2O 에서 쓰이는 Xgb의 하이퍼 파라미터가 다르다!
    # https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/xgboost.html?highlight=xgboost

    booster =  trial.suggest_categorical('booster', ['gbtree','dart'])# ,'dart'])
    eta = trial.suggest_uniform('eta', 1e-6, 0.3) # range : [0,1], [default=0.3] # learn_rate 와 같다.
    gamma = trial.suggest_uniform('gamma', 1e-6, 2) # Range: [0,∞], [default=0]

    max_depth = trial.suggest_int("max_depth", 4, 18, step=2) # range: [0,∞], [default=6]
    min_child_weight = trial.suggest_uniform("min_child_weight", 1, int(len(h2train_feat_1)/100)) # range: [0,∞], [default=1]
    max_delta_step = trial.suggest_uniform("max_delta_step", 1e-6, 2) # range: [0,∞], [default=0]

    sample_type  =  trial.suggest_categorical('sample_type', ['uniform','weighted'])
    max_bins =  trial.suggest_int("max_bins", 256, 1024+512, step=128) 
    max_leaves = trial.suggest_int("max_leaves", 0, 128, step=2) 
    # scale_pos_weight = trial.suggest_float("subsample", 0.1, 10, step=0.1)# Range: [0,∞], [default=1] # h2o 에 없다.
    
    ntrees =  trial.suggest_int("ntrees", 100, 1000, step=100) 
    tree_method = trial.suggest_categorical('tree_method', ['hist'])# ,'auto']) # auto : xgboost, hist : lgbm
    # hist & loss guide는 한 세트
    grow_policy = trial.suggest_categorical('grow_policy', ['lossguide'])#,'depthwise', ]) # depthwise : xgb, lossguide : Lgbm

    reg_lambda = trial.suggest_uniform('reg_lambda', 1e-6, 2) # Range: [0,∞], [default=1]
    reg_alpha = trial.suggest_uniform('reg_alpha', 1e-6, 2) # Range: [0,∞], [default=0]
    max_abs_leafnode_pred = trial.suggest_loguniform("max_abs_leafnode_pred", 1e-10, 5)
    
    subsample = trial.suggest_float("subsample", 0.1, 1, step=0.1) # range: (0,1], [default=1]
    colsample_bytree = trial.suggest_float("colsample_bytree", 0.5, 1, step=0.05)
    col_sample_rate_per_tree = trial.suggest_float("col_sample_rate_per_tree", 0.5, 1, step=0.05)
    # colsample_bynode = trial.suggest_float("colsample_bynode", 0.5, 1, step=0.05) # - H2O ver 차이로 인해 인식을 못하는듯
    col_sample_rate  = trial.suggest_float("col_sample_rate", 0.5, 1, step=0.05)
    
    seed = 42

    xgb_params = dict(booster=booster, 
                      eta=eta, 
                      gamma=gamma, 

                      max_depth=max_depth, 
                      min_child_weight=min_child_weight, 
                      max_delta_step=max_delta_step,

                      sample_type=sample_type,
                      max_bins =  max_bins,
                      max_leaves = max_bins,
                      # scale_pos_weight = scale_pos_weight,

                      ntrees=ntrees,
                      tree_method=tree_method,
                      grow_policy=grow_policy,

                      reg_lambda=reg_lambda, 
                      reg_alpha=reg_alpha, 
                      max_abs_leafnode_pred=max_abs_leafnode_pred,
                      
                      subsample=subsample, 
                      colsample_bytree=colsample_bytree, 
                      col_sample_rate_per_tree = col_sample_rate_per_tree,
                      # colsample_bynode=colsample_bynode, # - H2O ver 차이로 인해 인식을 못하는듯
                      col_sample_rate=col_sample_rate,
                      
                      seed=seed,
                      
                      stopping_rounds=400,
                      stopping_tolerance=0.00001,
                      stopping_metric="AUC", # AUC, logloss

                      # verbose=0,
                     )
    
    CV_score_array =[]
    #################################################################
    #### 에러의 주범
    predictors = list(h2train_feat_1.columns)
    numeric_response = 'TARGET'
    ##################################################################
    ####
    for i in range(n_splits):
      start_time = time.time()

    
      globals()['h2xgb_'+str(i+1)] = H2OXGBoostEstimator(**xgb_params )
      print('='*10,'h2xgb_'+str(i+1),'='*10)
      globals()['h2xgb_'+str(i+1)].train(x= predictors,
                                        y= numeric_response, 
                                        training_frame= globals()['h2train_xy_'+str(i+1)],
                                        validation_frame = globals()['h2valid_xy_'+str(i+1)]  
                                        )
      CV_score_array.append(globals()['h2xgb_'+str(i+1)].auc(valid=True)) # auc, logloss
      # H2O Criteria 보고서 적절히 auc ~~ 기타로 변경
      # https://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/xgboost.html?highlight=xgboost
      print("time :", time.time() - start_time)  # 현재시각 - 시작시간 = 실행 시간

    avg = np.mean(CV_score_array)
    return avg

In [ ]:
%%time
import joblib
from optuna.samplers import TPESampler

sampler = TPESampler(seed=42, n_ei_candidates=256*16, n_startup_trials=128*4) #, multivariate=True, group=True) # optuna-2.8.0
# https://optuna.readthedocs.io/en/stable/reference/generated/optuna.samplers.TPESampler.html

start3 = time.time()

xgb_study = optuna.create_study(direction="maximize", study_name='Xgb optimization',sampler=sampler) # maximize, minimize

xgb_study.optimize(Objective, timeout=3600*16) 
# https://optuna.readthedocs.io/en/stable/reference/generated/optuna.study.Study.html#optuna.study.Study.optimize
# timeout 옵션 존재 : timeout = n_sec

print("Total time :", time.time() - start3)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:
for i in range(5):
    start_time = time.time()
    predictors = list(h2train_feat_1.columns)
    numeric_response = 'TARGET'
    

    globals()['h2xgb_'+str(i+1)] = H2OXGBoostEstimator(**xgb_study.best_params ) # xgb_study.best_params
    print('='*10,'h2xgb_'+str(i+1),'='*10)
    globals()['h2xgb_'+str(i+1)].train(x= predictors,
                                    y= numeric_response, 
                                    training_frame= globals()['h2train_xy_'+str(i+1)],
                                    validation_frame = globals()['h2valid_xy_'+str(i+1)]  
                                    )

    print("time :", time.time() - start_time)  # 현재시각 - 시작시간 = 실행 시간

In [ ]:
for i in range(n_splits):
  print(i+1)
  print(globals()['h2xgb_'+str(i+1)].confusion_matrix(valid=True))

In [ ]:
perf = h2xgb_4.model_performance(valid=True)
perf

In [ ]:
xgb_study.best_params

In [ ]:
visualization.plot_optimization_history(xgb_study)

In [ ]:
fig = optuna.visualization.plot_parallel_coordinate(xgb_study)
fig.show()